In [3]:
import cv2

In [43]:
import cv2
import numpy as np

logo = cv2.imread("./image.png", cv2.IMREAD_GRAYSCALE)
shirt = cv2.imread("./shirts/image.png", cv2.IMREAD_GRAYSCALE)

if logo is None or shirt is None:
    print("Error loading images")
    exit()

# SIFT with your custom params
sift = cv2.SIFT_create(
    nOctaveLayers=5,
    sigma=1.414
)

kp_logo, des_logo = sift.detectAndCompute(logo, None)
kp_shirt, des_shirt = sift.detectAndCompute(shirt, None)

# --- 1) Match with KNN (no crossCheck here) ---
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
knn_matches = bf.knnMatch(des_logo, des_shirt, k=2)

# --- 2) Lowe's ratio test ---
good = []
ratio_thresh = 0.75
for m, n in knn_matches:
    if m.distance < ratio_thresh * n.distance:
        good.append(m)

print("Good matches:", len(good))

MIN_MATCH_COUNT = 0  # you can tweak this

if len(good) >= MIN_MATCH_COUNT:
    # --- 3) Prepare points for findHomography ---
    src_pts = np.float32([kp_logo[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp_shirt[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

    # --- 4) Compute Homography using RANSAC ---
    H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    matchesMask = mask.ravel().tolist() if mask is not None else None

    print("Homography:\n", H)

    # --- 5) Draw inlier matches only ---
    result = cv2.drawMatches(
        logo, kp_logo,
        shirt, kp_shirt,
        good, None,
        matchesMask=matchesMask,
        flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
    )

    cv2.namedWindow("RANSAC Matches", cv2.WINDOW_NORMAL)
    cv2.imshow("RANSAC Matches", result)
    cv2.resizeWindow("RANSAC Matches", 1200, 800)

    # --- 6) Optional: draw logo projection (bounding box) on shirt ---
    if H is not None:
        h, w = logo.shape
        # Logo corners
        logo_corners = np.float32([
            [0, 0],
            [0, h - 1],
            [w - 1, h - 1],
            [w - 1, 0]
        ]).reshape(-1, 1, 2)

        # Project corners into shirt image
        projected_corners = cv2.perspectiveTransform(logo_corners, H)

        # Convert shirt to color to draw colored polygon
        shirt_color = cv2.cvtColor(shirt, cv2.COLOR_GRAY2BGR)
        shirt_with_box = cv2.polylines(
            shirt_color,
            [np.int32(projected_corners)],
            isClosed=True,
            color=(0, 255, 0),
            thickness=3,
            lineType=cv2.LINE_AA
        )

        cv2.namedWindow("Logo on Shirt", cv2.WINDOW_NORMAL)
        cv2.imshow("Logo on Shirt", shirt_with_box)
        cv2.resizeWindow("Logo on Shirt", 800, 800)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

else:
    print(f"Not enough matches for RANSAC: {len(good)} / {MIN_MATCH_COUNT}")


Good matches: 16
Homography:
 [[ 8.28444736e-01 -1.49648199e+00  2.01712244e+02]
 [ 3.15802324e-01 -8.96780058e-01  2.30443087e+02]
 [ 1.59485899e-03 -5.06620221e-03  1.00000000e+00]]


In [ ]:
import cv2
import numpy as np

# Read image
img = cv2.imread("./R.jpeg")
Z = img.reshape((-1, 3))          # shape: (num_pixels, 3)
Z = np.float32(Z)

# Define criteria and number of clusters (K)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
K = 4  # number of segments
attempts = 10


filtered = cv2.pyrMeanShiftFiltering(img, sp=20, sr=40)

cv2.imshow("Original", img)
cv2.imshow("MeanShift Filtered", filtered)
cv2.waitKey(0)
cv2.destroyAllWindows()

gray = cv2.cvtColor(filtered, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)

cv2.imshow("Threshold after MeanShift", thresh)0
cv2.waitKey(0)
cv2.destroyAllWindows()

Good matches: 6
Not enough matches for RANSAC: 6 / 10


In [ ]:
import cv2
import numpy as np

# ---------- 1) Load images ----------
logo = cv2.imread("./licensed-image.jpeg", cv2.IMREAD_GRAYSCALE)

# Read shirt in color (for segmentation) and gray (for SIFT)
shirt_color = cv2.imread("./shirts/image.png")
shirt_gray = cv2.cvtColor(shirt_color, cv2.COLOR_BGR2GRAY)

if logo is None or shirt_color is None:
    print("Error loading images")
    exit()

# ---------- 2) Segment shirt with MeanShift + threshold ----------
# MeanShift smoothing (color-based segmentation)
filtered = cv2.pyrMeanShiftFiltering(shirt_color, sp=10, sr=10)

# Convert to gray and threshold (Otsu) to get binary mask
filtered_gray = cv2.cvtColor(filtered, cv2.COLOR_BGR2GRAY)
_, mask = cv2.threshold(filtered_gray, 0, 255, cv2.THRESH_OTSU)

# Optional: clean mask a bit with morphology
kernel = np.ones((3, 3), np.uint8)
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=1)

cv2.imshow("MeanShift Filtered", filtered)
cv2.imshow("Segmentation Mask", mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

# ---------- 3) SIFT with custom params ----------
sift = cv2.SIFT_create(
    nOctaveLayers=7,
    sigma=1.414
)

# Logo: no mask
kp_logo, des_logo = sift.detectAndCompute(logo, None)

# Shirt: use segmentation mask to restrict keypoints
kp_shirt, des_shirt = sift.detectAndCompute(shirt_gray, mask)

if des_logo is None or des_shirt is None:
    print("No descriptors found – maybe the mask removed too much?")
    exit()

# ---------- 4) KNN matching + Lowe's ratio ----------
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
knn_matches = bf.knnMatch(des_logo, des_shirt, k=2)

good = []
ratio_thresh = 0.75
for m, n in knn_matches:
    if m.distance < ratio_thresh * n.distance:
        good.append(m)

print("Good matches:", len(good))

MIN_MATCH_COUNT = 0  # tweak as you like

if len(good) >= MIN_MATCH_COUNT:
    # ---------- 5) Prepare points for RANSAC homography ----------
    src_pts = np.float32([kp_logo[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp_shirt[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

    # ---------- 6) Compute homography with RANSAC ----------
    H, mask_inliers = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    matchesMask = mask_inliers.ravel().tolist() if mask_inliers is not None else None

    print("Homography:\n", H)

    # ---------- 7) Draw inlier matches ----------
    result = cv2.drawMatches(
        logo, kp_logo,
        shirt_gray, kp_shirt,
        good, None,
        matchesMask=matchesMask,
        flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
    )

    cv2.namedWindow("RANSAC Matches (Segmented Shirt)", cv2.WINDOW_NORMAL)
    cv2.imshow("RANSAC Matches (Segmented Shirt)", result)
    cv2.resizeWindow("RANSAC Matches (Segmented Shirt)", 1200, 800)

    # ---------- 8) Project logo bounding box onto shirt ----------
    if H is not None:
        h_logo, w_logo = logo.shape

        logo_corners = np.float32([
            [0, 0],
            [0, h_logo - 1],
            [w_logo - 1, h_logo - 1],
            [w_logo - 1, 0]
        ]).reshape(-1, 1, 2)

        projected_corners = cv2.perspectiveTransform(logo_corners, H)

        # Draw on color shirt image
        shirt_with_box = shirt_color.copy()
        shirt_with_box = cv2.polylines(
            shirt_with_box,
            [np.int32(projected_corners)],
            isClosed=True,
            color=(0, 255, 0),
            thickness=3,
            lineType=cv2.LINE_AA
        )

        cv2.namedWindow("Logo on Segmented Shirt", cv2.WINDOW_NORMAL)
        cv2.imshow("Logo on Segmented Shirt", shirt_with_box)
        cv2.resizeWindow("Logo on Segmented Shirt", 800, 800)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

else:
    print(f"Not enough matches for RANSAC: {len(good)} / {MIN_MATCH_COUNT}")


Good matches: 12
Homography:
 [[-1.06748192e-01 -3.60462436e-01  9.76273172e+02]
 [ 2.60882446e-01  2.46112121e-01 -5.76474955e+02]
 [ 1.82703993e-04 -2.76827660e-04  1.00000000e+00]]
